# Libraries

In [1]:
from  google.colab  import  drive 
drive.mount ('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import scipy
import math
 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,balanced_accuracy_score
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics.pairwise as pw
from sklearn.metrics.pairwise import pairwise_distances
from mlxtend.evaluate import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import confusion_matrix


import matplotlib.pyplot as plt  
%matplotlib inline    
import matplotlib as mpl
import seaborn as sns

import random
from scipy import sparse
from scipy.stats import pearsonr
import re

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

import nltk
from nltk.corpus import stopwords

import torch
from torchtext import data
from torchtext import datasets

# Recommend with SVD and Emotions

## Emotion Analysis

Εξάγουμε το συναίσθημα από το input του χρήστη, και βρίσκουμε τα εστιατόρια που προσφέρουν αυτό το συναίσθημα περισσότερο.

In [ ]:
df_emotions = pd.read_csv('/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Restaurants Reviews/reviews_with_emotions_full.csv')
df_emotions.drop(["Unnamed: 0"], axis = 1, inplace=True)

In [ ]:
username = input ("Username: ")
password = input ("Password: ")

Username: 1000
Password: 2


In [ ]:
users_comments = df_emotions[df_emotions['UserId'] == int(username)]

In [ ]:
user_positive_comments = users_comments[users_comments['Rating'] > 3]

In [ ]:
already_visited = users_comments["Name"].values

In [ ]:
user_positive_comments = user_positive_comments.groupby("Emotion").count()
user_positive_comments = user_positive_comments.sort_values(by=['Emotion'], ascending = True)

In [ ]:
most_felt_emotion = user_positive_comments.index.values[0]

In [ ]:
possible_options = df_emotions[df_emotions['Emotion'] == most_felt_emotion]

In [ ]:
recommends_df = possible_options.groupby("Name")['Emotion'].count().to_frame()
recommends_df = recommends_df.sort_values(by=['Emotion'], ascending = False)

In [ ]:
possible_recommendations = recommends_df.index.values

In [ ]:
recommendations_from_emotion = []
for i in possible_recommendations:
  if i not in already_visited:
    recommendations_from_emotion.append(i)

## SVD Implementation

In [3]:
df = pd.read_csv('/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Restaurants Reviews/restaurants_for_SVD.csv')

In [4]:
df

,Unnamed: 0,Name,UserId,Review,Rating,sentiment,Review_Clean,Reviewer_Count,Restaurant_Count
0,0,aleria,1,a full 5 stars! phenomenal michelin ranked res...,5,2,a full 5 stars phenomenal michelin ranked rest...,30,886
1,1,lithos,285,"amazing food, amazing service! a place that wi...",5,2,amazing food amazing service a place that will...,30,1005
2,2,lithos,257,great restaurant with delicious food and great...,5,2,great restaurant with delicious food and great...,30,1005
3,3,lithos,258,"great food and great service minded staff, hig...",5,2,great food and great service minded staff high...,30,1005
4,4,lithos,259,we visited this restaurant in july 2021. we we...,5,2,we visited this restaurant in july 2021 we wer...,30,1005
...,...,...,...,...,...,...,...,...,...
26203,26203,gods_restaurant,1005,"xcellent food, had a chicken souvlaki and grea...",5,2,xcellent food had a chicken souvlaki and great...,11,1005
26204,26204,karamanlidika,1005,this place is spectacular. recommended by two...,5,2,this place is spectacular recommended by two ...,11,1005
26205,26205,efcharis,1005,we visited efxaris for lunch. great lamb shop...,4,2,we visited efxaris for lunch great lamb shops...,11,1005
26206,26206,oroscopo,1005,my 11 year old son picked this restaurant befo...,5,2,my 11 year old son picked this restaurant befo...,11,1005


In [ ]:
pivot = pd.pivot_table(df, index='UserId', columns=['Name'], values='Rating').fillna(0)
pivot.head(5)

Name,aleria,alexander_the_great,arcadia,athenascook,atitamos,balcony,dio_dekares,efcharis,elaea,gods_restaurant,...,rozalia,scholarchio,sin_athina,smile,strofi,the_greco,to_kati_allo,to_paradosiako,tzitzikas_kai_mermigas,ydria
UserId,,,,,,,,,,,,,,,,,,,,,
1,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,1.0,5.0,3.0,4.0,4.0,5.0,5.0,4.0
2,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,4.0,5.0,5.0,2.0,5.0,4.0,5.0
3,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,4.0,1.0,5.0,5.0,5.0,5.0,5.0,4.0,1.0
4,5.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,3.0
5,5.0,5.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,4.0,3.0


In [ ]:
pivot_mat = pivot.values
pivot_mat[:5]

array([[5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
        4., 4., 4., 5., 5., 5., 1., 5., 3., 4., 4., 5., 5., 4.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 4., 3., 5., 5., 4., 5.,
        5., 4., 5., 5., 5., 5., 5., 4., 5., 5., 2., 5., 4., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 4., 5.,
        5., 4., 3., 5., 5., 4., 1., 5., 5., 5., 5., 5., 4., 1.],
       [5., 3., 5., 5., 5., 5., 5., 5., 5., 5., 5., 2., 5., 5., 5., 3.,
        3., 5., 4., 5., 4., 5., 5., 5., 5., 5., 5., 5., 4., 3.],
       [5., 5., 3., 5., 5., 5., 5., 5., 5., 5., 5., 1., 5., 5., 5., 5.,
        5., 5., 4., 5., 5., 5., 5., 4., 5., 5., 5., 5., 4., 3.]])

In [ ]:
user_id = list(pivot.index)
user_id[:10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
sparse_matrix = csr_matrix(pivot_mat)
sparse_matrix

<1005x30 sparse matrix of type '<class 'numpy.float64'>'
	with 26208 stored elements in Compressed Sparse Row format>

In [ ]:
factor_n = 15
U, sigma, V = svds(sparse_matrix, k = factor_n)

In [ ]:
print(U.shape)
print(V.shape)
sigma = np.diag(sigma)
print(sigma.shape)

(1005, 15)
(15, 30)
(15, 15)


In [ ]:
pred_rating = np.dot(np.dot(U,sigma),V)
pred_rating[:5]

array([[5.10613796, 4.48722673, 4.73475454, 4.80621782, 4.88183062,
        4.99760712, 4.92093591, 4.9189482 , 4.55732936, 5.12886634,
        4.69241383, 4.80996617, 4.67473774, 4.74052777, 4.58930268,
        4.79756014, 4.7088804 , 3.3781504 , 4.13234416, 4.7511676 ,
        4.98197478, 4.7635279 , 1.48021573, 5.29080624, 3.64439301,
        4.45698935, 5.14757571, 5.13658934, 5.19350454, 4.23808976],
       [5.1780047 , 4.79531736, 4.62255726, 4.68738802, 4.66093946,
        4.79319687, 5.27134501, 4.69743762, 4.81049355, 3.94860267,
        4.6791785 , 2.87129099, 4.82829151, 4.75691864, 4.73641954,
        4.96269761, 5.08144633, 4.45012101, 5.23501248, 4.88282733,
        4.86026773, 5.19317462, 4.85309023, 3.90368325, 4.69832569,
        4.97114123, 3.79886537, 4.14528175, 4.65327412, 4.93716716],
       [5.03434556, 4.89957515, 4.81076389, 4.90233446, 4.69194523,
        4.88285433, 5.20512773, 4.92412185, 4.57900452, 4.80572762,
        4.71725059, 4.95058505, 4.99823657, 4.

In [ ]:
pred_rating_n = (pred_rating - pred_rating.min()) / (pred_rating.max() - pred_rating.min())

In [ ]:
pred_rating_n[:5]

array([[0.8407907 , 0.7656044 , 0.79567447, 0.80435594, 0.8135415 ,
        0.8276062 , 0.81829207, 0.8180506 , 0.77412058, 0.84355178,
        0.79053085, 0.80481129, 0.78838353, 0.79637581, 0.77800475,
        0.80330419, 0.79253124, 0.63087209, 0.72249272, 0.79766835,
        0.82570716, 0.7991699 , 0.40030805, 0.86322449, 0.66321566,
        0.76193112, 0.84582462, 0.84448998, 0.85140413, 0.73533886],
       [0.84952118, 0.80303173, 0.78204457, 0.78992031, 0.7867073 ,
        0.80277413, 0.86086031, 0.79114115, 0.80487536, 0.70017152,
        0.788923  , 0.56929803, 0.80703748, 0.798367  , 0.79587673,
        0.82336534, 0.83779112, 0.76109675, 0.85644658, 0.81366258,
        0.810922  , 0.85136405, 0.81005007, 0.69471464, 0.79124903,
        0.82439109, 0.6819812 , 0.7240644 , 0.7857761 , 0.82026387],
       [0.83206925, 0.81569713, 0.8049082 , 0.81603234, 0.79047393,
        0.81366586, 0.85281613, 0.8186791 , 0.77675371, 0.80429639,
        0.79354806, 0.82189389, 0.82768267, 0.

In [ ]:
pred_df = pd.DataFrame(pred_rating_n, columns = pivot.columns, index=user_id).transpose()
pred_df.head(15)

,1,2,3,4,5,6,7,8,9,10,...,996,997,998,999,1000,1001,1002,1003,1004,1005
Name,,,,,,,,,,,,,,,,,,,,,
aleria,0.840791,0.849521,0.832069,0.826586,0.829534,0.850700,0.826101,0.821303,0.803865,0.608446,...,0.175666,0.170026,0.188437,0.169834,0.204087,0.174247,0.188270,0.182627,0.211295,0.228816
alexander_the_great,0.765604,0.803032,0.815697,0.539607,0.788292,0.772540,0.808357,0.670405,0.750106,0.815918,...,0.236909,0.223250,0.213933,0.243060,0.200962,0.210047,0.236527,0.225588,0.229142,0.230917
arcadia,0.795674,0.782045,0.804908,0.806817,0.793407,0.755325,0.777514,0.778265,0.783536,0.793633,...,0.774586,0.781862,0.760503,0.724855,0.693833,0.721865,0.763123,0.770282,0.682637,0.678879
athenascook,0.804356,0.789920,0.816032,0.826216,0.827099,0.825129,0.828785,0.600753,0.832912,0.794698,...,0.279450,0.273371,0.247500,0.286322,0.244472,0.279313,0.271596,0.272690,0.222515,0.211629
atitamos,0.813541,0.786707,0.790474,0.817733,0.801028,0.786946,0.814760,0.846698,0.809937,0.708314,...,0.421213,0.426928,0.400100,0.404514,0.336021,0.359900,0.381342,0.377768,0.332380,0.320442
balcony,0.827606,0.802774,0.813666,0.833192,0.828836,0.784506,0.814459,0.862778,0.825778,0.728622,...,0.371548,0.395491,0.375226,0.386707,0.321844,0.344877,0.356613,0.366912,0.327815,0.318894
dio_dekares,0.818292,0.860860,0.852816,0.777990,0.865087,0.768681,0.726027,0.791218,0.764605,0.810606,...,0.225375,0.226762,0.233611,0.215141,0.228063,0.212754,0.215067,0.217100,0.231729,0.241280
efcharis,0.818051,0.791141,0.818679,0.831143,0.814767,0.797667,0.777961,0.826791,0.794893,0.775168,...,0.800941,0.783022,0.769021,0.706325,0.716507,0.735800,0.780836,0.776106,0.693774,0.707311
elaea,0.774121,0.804875,0.776754,0.831122,0.814617,0.764801,0.830827,0.841811,0.810046,0.717855,...,0.524922,0.543284,0.492267,0.505442,0.387169,0.419060,0.454442,0.450911,0.387066,0.365706


In [ ]:
def recommend_restaurants(user_id ,pred_df, items_df, restaurants_to_ignore=[], top_list=20, verbose=False):

        sorted_user_predictions = pred_df[user_id].sort_values(ascending=False).reset_index().rename(columns={user_id: 'recStrength'})

        recommendations_df = sorted_user_predictions[~sorted_user_predictions['Name'].isin(restaurants_to_ignore)].sort_values(by='recStrength', ascending = False).head(top_list)
        print(recommendations_df)
        return recommendations_df


In [ ]:
def recommender(user_id,pred_df,real_df,df_meta):

    restaurants_to_ignore = df[df['UserId'] == str(user_id)]['Name'].values
    recommend = recommend_restaurants(user_id, pred_df,real_df, restaurants_to_ignore)
    df_user= real_df.loc[real_df['UserId'] == user_id]
    new_df = df_user.merge(recommend, how = 'outer', left_on = 'Name', right_on = 'Name')
    rec_df = new_df.loc[new_df['Rating'].isnull()==True]
    df_rec_t = rec_df.loc[:, ['Name', 'recStrength']]
    df_last = pd.merge(df_rec_t,df_meta[['RestaurantId','Name']],on=['Name'], how='left')
    df_last = df_last.drop_duplicates(subset=["Name", "RestaurantId"], keep='first')
    df_last = df_last.reset_index(drop=True) 
    return df_last

In [ ]:
restaurants = pd.read_csv('/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Restaurants Reviews/restaurants.csv')
ratings = pd.read_csv('/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Restaurants Reviews/ratings.csv')
df_meta = restaurants.merge(ratings, how="left", on="RestaurantId")
df_meta = df_meta.astype({"UserId": str, "RestaurantId": str})
df_meta.head()

,RestaurantId,Name,UserId,Review,Rating
0,1,aleria,1,A full 5 stars! Phenomenal Michelin ranked res...,5
1,1,aleria,2,We spent great evening here. Cozy restaurant w...,5
2,1,aleria,3,10/10. Aleria was our highlight during our tri...,5
3,1,aleria,4,Great service and fantastic food. We enjoyed e...,5
4,1,aleria,5,Let me put it his way: simply perfect. We hate...,5


In [ ]:
df = df.astype({"UserId": str})
df

,Unnamed: 0,Name,UserId,Review,Rating,sentiment,Review_Clean,Reviewer_Count,Restaurant_Count
0,0,aleria,1,a full 5 stars! phenomenal michelin ranked res...,5,2,a full 5 stars phenomenal michelin ranked rest...,30,886
1,1,lithos,285,"amazing food, amazing service! a place that wi...",5,2,amazing food amazing service a place that will...,30,1005
2,2,lithos,257,great restaurant with delicious food and great...,5,2,great restaurant with delicious food and great...,30,1005
3,3,lithos,258,"great food and great service minded staff, hig...",5,2,great food and great service minded staff high...,30,1005
4,4,lithos,259,we visited this restaurant in july 2021. we we...,5,2,we visited this restaurant in july 2021 we wer...,30,1005
...,...,...,...,...,...,...,...,...,...
26203,26203,gods_restaurant,1005,"xcellent food, had a chicken souvlaki and grea...",5,2,xcellent food had a chicken souvlaki and great...,11,1005
26204,26204,karamanlidika,1005,this place is spectacular. recommended by two...,5,2,this place is spectacular recommended by two ...,11,1005
26205,26205,efcharis,1005,we visited efxaris for lunch. great lamb shop...,4,2,we visited efxaris for lunch great lamb shops...,11,1005
26206,26206,oroscopo,1005,my 11 year old son picked this restaurant befo...,5,2,my 11 year old son picked this restaurant befo...,11,1005


In [ ]:
recommendations_from_SVD = recommender(int(username),pred_df,df,df_meta)

                      Name  recStrength
13                   elaea     0.387169
14                atitamos     0.336021
15                 balcony     0.321844
16                   nolan     0.314612
17               mani_mani     0.252015
18             athenascook     0.244472
19             scholarchio     0.235359
20             dio_dekares     0.228063
21              klimataria     0.218106
22               the_greco     0.217771
23                  mironi     0.212164
24                  aleria     0.204087
25     alexander_the_great     0.200962
26  tzitzikas_kai_mermigas     0.196950
27                 rozalia     0.190623
28              sin_athina     0.144441
29          to_paradosiako     0.007869


## Combinign SVD and Emotion Analysis Results

Τώρα, συνδυάζουμε τα recommendations από τις 2 μεθόδους.
Αυτό γίνεται αυξάνοντας το strength που προέρχεται από το SVD, κατά ένα ποσοστό αναλόγως το συναίσθημα που προσφέρει το κάθε εστιατόριο.
Για παράδειγμα, για το πρώτο εστιατόριο που προτείνεται με βάση το συναίσθημα, αυξάνουμε το strength κατά 20%. Για το δεύτερο, το αυξάνουμε κατά 10% ενώ το τρίτο κατά 5%.

In [ ]:
recommendations_from_emotion

['balcony',
 'elaea',
 'sin_athina',
 'aleria',
 'to_paradosiako',
 'nolan',
 'tzitzikas_kai_mermigas',
 'mironi',
 'alexander_the_great',
 'scholarchio',
 'klimataria',
 'rozalia',
 'atitamos',
 'mani_mani',
 'athenascook']

In [ ]:
recommendations_from_SVD

,Name,recStrength,RestaurantId
0,elaea,0.387169,9
1,atitamos,0.336021,5
2,balcony,0.321844,6
3,nolan,0.314612,18
4,mani_mani,0.252015,16
5,athenascook,0.244472,4
6,scholarchio,0.235359,22
7,dio_dekares,0.228063,7
8,klimataria,0.218106,12
9,the_greco,0.217771,26


In [ ]:
score_dict = {}

for index, row in recommendations_from_SVD.iterrows():
  name = row['Name']
  print("Name is ", name)
  recStrength = row['recStrength']
  print("recStrength is ", recStrength)
  if(len(recommendations_from_emotion) >= 3):
    if(name == recommendations_from_emotion[0]):
        print('1')
        recStrength = recStrength + 0.2*recStrength
        print("new recStrength is ", recStrength)
    elif(name == recommendations_from_emotion[1]):
        recStrength = recStrength + 0.1*recStrength
        print("new recStrength is ", recStrength)
    elif(name == recommendations_from_emotion[2]):
        recStrength = recStrength + 0.05*recStrength
        print("new recStrength is ", recStrength)
  elif(len(recommendations_from_emotion) == 2):
    if(name == recommendations_from_emotion[0]):  
      recStrength = recStrength + 0.2*recStrength
      print("new recStrength is ", recStrength)
    elif(name == recommendations_from_emotion[1]):
      recStrength = recStrength + 0.1*recStrength
      print("new recStrength is ", recStrength)
  elif(len(recommendations_from_emotion) == 1):
    if(name == recommendations_from_emotion[0]):
      recStrength = recStrength + 0.2*recStrength
      print("new recStrength is ", recStrength)
  print('\n')
  score_dict[index] = recStrength    


Name is  elaea
recStrength is  0.3871691038463119
new recStrength is  0.42588601423094313


Name is  atitamos
recStrength is  0.336021441198229


Name is  balcony
recStrength is  0.32184395255325327
1
new recStrength is  0.3862127430639039


Name is  nolan
recStrength is  0.31461191199184035


Name is  mani_mani
recStrength is  0.2520147202543976


Name is  athenascook
recStrength is  0.24447239611603228


Name is  scholarchio
recStrength is  0.23535896230905995


Name is  dio_dekares
recStrength is  0.2280634059429857


Name is  klimataria
recStrength is  0.2181055242162521


Name is  the_greco
recStrength is  0.21777133193551887


Name is  mironi
recStrength is  0.21216384975149016


Name is  aleria
recStrength is  0.204086862691315


Name is  alexander_the_great
recStrength is  0.20096202356847268


Name is  tzitzikas_kai_mermigas
recStrength is  0.19694995248597447


Name is  rozalia
recStrength is  0.19062279512704983


Name is  sin_athina
recStrength is  0.14444143384819708
new r

In [ ]:
score_dict

{0: 0.42588601423094313,
 1: 0.336021441198229,
 2: 0.3862127430639039,
 3: 0.31461191199184035,
 4: 0.2520147202543976,
 5: 0.24447239611603228,
 6: 0.23535896230905995,
 7: 0.2280634059429857,
 8: 0.2181055242162521,
 9: 0.21777133193551887,
 10: 0.21216384975149016,
 11: 0.204086862691315,
 12: 0.20096202356847268,
 13: 0.19694995248597447,
 14: 0.19062279512704983,
 15: 0.15166350554060692,
 16: 0.007869078128647284}

In [ ]:
import operator
sorted_scores = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)
sorted_scores

[(0, 0.42588601423094313),
 (2, 0.3862127430639039),
 (1, 0.336021441198229),
 (3, 0.31461191199184035),
 (4, 0.2520147202543976),
 (5, 0.24447239611603228),
 (6, 0.23535896230905995),
 (7, 0.2280634059429857),
 (8, 0.2181055242162521),
 (9, 0.21777133193551887),
 (10, 0.21216384975149016),
 (11, 0.204086862691315),
 (12, 0.20096202356847268),
 (13, 0.19694995248597447),
 (14, 0.19062279512704983),
 (15, 0.15166350554060692),
 (16, 0.007869078128647284)]

In [ ]:
counter = 0
#δημιουργία νέου dataframe για την αποθήκευση των recommendations
recommendations = pd.DataFrame(columns=('Name', 'score'))
# τα reviews με τα υψηλότερα cosign similarity
for i in sorted_scores:
  recommendations = recommendations.append({'Name': recommendations_from_SVD.iloc[i[0]]['Name'],
                          'score': i[1]}, ignore_index=True)
            
  counter += 1
  if counter>10:
    break

In [ ]:
recommendations

,Name,score
0,elaea,0.425886
1,balcony,0.386213
2,atitamos,0.336021
3,nolan,0.314612
4,mani_mani,0.252015
5,athenascook,0.244472
6,scholarchio,0.235359
7,dio_dekares,0.228063
8,klimataria,0.218106
9,the_greco,0.217771
